## Import modules

In [1]:
import string
import re
import nltk

## Function : Create trigram dictionary

In [2]:
def add_trigrams(trigrams,trigram):
    for tri in trigrams:
        stri = ' '.join(tri)
        if not trigram or stri not in trigram:
            trigram[stri] = 1;
        else:
            trigram[stri] = trigram[stri] + 1

## Function : Create quadgram dictionary

In [3]:
def add_quadgrams(quadgrams,quadgram):
    for quad in quadgrams:
        stri = ' '.join(quad)
        if not quadgram or stri not in quadgram:
            quadgram[stri] = 1;
        else:
            quadgram[stri] = quadgram[stri] + 1

## Function : Create probability dictionary

In [4]:
def add_probability(trigram,quadgram,probab):
    for quad in quadgram:
        #print(quad)
        s = quad.split()
        #print(s)
        w = s[-1]
        del s[-1]
        stri = ' '.join(s)
        #print(stri)
        p = quadgram[quad]/trigram[stri]
        if stri in probab:
            if p > probab[stri][1]:
                probab[stri][0] = w
                probab[stri][1] = p
        else:
            probab.setdefault(stri,[])
            probab[stri].append(w)
            probab[stri].append(p)

## Function : Create vocabulary list

In [5]:
def create_token_list(tokens,vocab):
    newlist=[]
    for word in tokens:
        if word:
            if not vocab or word not in vocab: 
                vocab[word] = len(vocab)
            newlist.append(str(vocab[word]))
    return newlist

## Function : Remove punctuations

In [6]:
def remove_punc(text):
    for c in string.punctuation:
        if c!='\'':
            text=text.replace(c," ")
    text = text.replace("' "," ")
    text = text.replace("\n"," ")
    answer = text.lower()
    return answer

## Function : Tokenize the corpus

In [7]:
def tokenize(text, ngrams,vocab):
    clean_text = remove_punc(text)
    tokens = create_token_list(clean_text.split(),vocab)
    #print(tokens)
    return [tuple(tokens[i:i+ngrams]) for i in range(len(tokens)-ngrams+1)]

## Function : Predict word

In [8]:
def predictWord(s,probab,vocab):
    predict = ""
    if s in probab:
        predict = probab[s][0]
    return predict

## Preprocessing performed

In [13]:
def train_data(trigram,quadgram,vocab):
    l = 0
    with open('corpusfile1.txt',buffering=20000,encoding='latin1') as f:
        for line in f:
            data = tokenize(line,3,vocab)
            #print(data)
            add_trigrams(data,trigram)
            data = tokenize(line,4,vocab)
            l = l + len(data)
            add_quadgrams(data,quadgram)
    f.close()
    return l

## Function : Calculate score for model

In [10]:
def calculate_score(probab,vocab):
    score = 0
    l = list(vocab.keys())
    with open('testfile.txt',buffering=20000,encoding='latin1') as f:
        for line in f:
            text = remove_punc(line)
            tokens = text.split()
            data = [tuple(tokens[i:i+4]) for i in range(len(tokens)-4+1)]            
            for quad in data:
                numbers = []
                flag = 0
                for ele in quad:
                    if ele in vocab:
                        numbers.append(str(vocab[ele]))
                    else:
                        flag = 1
                        break
                if flag == 0:
                    tri = ' '.join(numbers).split()
                    w = tri[-1]
                    del tri[-1]
                    s = ' '.join(tri)
                    predict = predictWord(s,probab,vocab)
                    if w == predict:
                        score = score + 1
                        #print(score,',',w,',',predict)
                        #print(l[(int)(numbers[0])],' ',l[(int)(numbers[1])],' ',l[(int)(numbers[2])],' ',l[(int)(numbers[3])])
    f.close()
    print("Score : ",score)

## Function : Calculate Perplexity after Add-1 Smoothing

In [22]:
def calculate_Perplexity(n,vocab,trigram,quadgram):
    p = 1.0
    with open('testfile.txt',buffering=20000,encoding='latin1') as f:
        for line in f:
            text = remove_punc(line)
            tokens = text.split()
            data = [tuple(tokens[i:i+4]) for i in range(len(tokens)-4+1)] 
            for quad in data:
                count_quadgrams = 1
                count_trigrams = len(vocab)
                numbers = []
                flag = 0
                for ele in quad:
                    if ele in vocab:
                        numbers.append(str(vocab[ele]))
                    else:
                        flag = 1
                        break
                if flag == 0:
                    four = ' '.join(numbers)
                    three = ' '.join(numbers[0:3])
                    if three in trigram:
                        count_trigrams = count_trigrams + trigram[three]
                    if four in quadgram:
                        count_quadgrams = count_quadgrams + quadgram[four]
                p=p*(((1/float(count_quadgrams))*count_trigrams)**(1./n))
    print("Quadgram Perplexity : ",p)        
    

## Run the Language Model

In [24]:
%%timeit
def main():
    vocab = {}
    probab = {}
    trigram = {}
    quadgram = {}
    n = train_data(trigram,quadgram,vocab)
    #add_probability(trigram,quadgram,probab)
    #calculate_score(probab,vocab)
    calculate_Perplexity(n,vocab,trigram,quadgram)
if __name__== "__main__":
  main()

Quadgram Perplexity :  2.2407648913689933
Quadgram Perplexity :  2.2407648913689933
Quadgram Perplexity :  2.2407648913689933
Quadgram Perplexity :  2.2407648913689933
1 loop, best of 3: 12.5 s per loop
